In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from unidecode import unidecode

distance=pd.read_excel("Turkish network.xls",sheet_name="Distance (km)", skiprows=[0], index_col=1).iloc[:,1:]
travel_time=pd.read_excel("Turkish network.xls",sheet_name="Travel time (min)", skiprows=[0], index_col=1).iloc[:,1:]
flow = pd.read_excel("Turkish network.xls",sheet_name="Flow", skiprows=[0], index_col=1).iloc[:,1:]
fixed_link_cost = pd.read_excel("Turkish network.xls",sheet_name="Fixed link cost", skiprows=[0], index_col=1).iloc[:,1:]
fixed_hub_cost = pd.read_excel("Turkish network.xls", sheet_name="Fixed hub cost", index_col=1).iloc[:,1:]

cities = list(distance.columns)
cities = [unidecode(city) for city in cities]

distance.index = cities
distance.columns = cities

travel_time.index = cities
travel_time.columns = cities

flow.index = cities
flow.columns = cities

fixed_link_cost.index = cities
fixed_link_cost.columns = cities

fixed_hub_cost.index = cities

ModuleNotFoundError: No module named 'gurobipy'

In [2]:
fixed_hub_cost = dict(zip(cities, fixed_hub_cost.iloc[:,0]))
alpha = 0.5

In [3]:
def floyd(G):
    nV = len(G)
    dist = list(map(lambda p: list(map(lambda q: q, p)), G))

    # Adding vertices individually
    for r in range(nV):
        for p in range(nV):
            for q in range(nV):
                dist[p][q] = min(dist[p][q], dist[p][r] + dist[r][q])
    return dist

In [4]:
u = distance * fixed_link_cost / 1000
u_hub = alpha * pd.DataFrame(floyd(np.array(u)), index=cities, columns=cities)
possible_hubs = cities
o = np.sum(flow, axis=1)
o = dict(zip(o.index, o.values))

#### Solution adapted from:
Andreas T. Ernst, Mohan Krishnamoorthy,
Exact and heuristic algorithms for the uncapacitated multiple allocation p-hub median problem,
European Journal of Operational Research,
Volume 104, Issue 1,
1998,
Pages 100-112,
ISSN 0377-2217,
https://doi.org/10.1016/S0377-2217(96)00340-2.
(https://www.sciencedirect.com/science/article/pii/S0377221796003402)

### Solution without shortest paths

In [5]:
#model
m = gp.Model("q2_part2")
m.ModelSense = GRB.MINIMIZE
#decision variables 
h=m.addVars(possible_hubs, name="h", vtype=GRB.BINARY)
x=m.addVars(cities, cities, possible_hubs, name="x",vtype=GRB.CONTINUOUS, lb = 0)
y=m.addVars(cities, possible_hubs, possible_hubs, name="y", vtype=GRB.CONTINUOUS, lb = 0)
z=m.addVars(cities, possible_hubs, name="z", vtype=GRB.CONTINUOUS, lb = 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-08


In [6]:
hub_limit=m.addConstr(gp.quicksum(fixed_hub_cost[i]*h[i] for i in possible_hubs) <= 2400,"hub_cost_limit")

cons1 = m.addConstrs(
    (gp.quicksum(z[(i,k)] for k in possible_hubs) == o[i] for i in cities))

cons2 = m.addConstrs(
    (gp.quicksum(x[(i,j,l)] for l in possible_hubs) == flow.at[i,j] for i in cities for j in cities))

cons3 = m.addConstrs(
    (gp.quicksum(y[(i,k,l)] for l in possible_hubs) + 
     gp.quicksum(x[(i,j,k)] for j in cities) - 
     gp.quicksum(y[(i,l,k)] for l in possible_hubs) -
     z[(i,k)] == 0 for i in cities for k in possible_hubs))

cons4 = m.addConstrs(
    (z[(i,k)] <= o[i] * h[k] for i in cities for k in possible_hubs))

cons5 = m.addConstrs(
    (x[i,j,l] <= flow.at[i,j] * h[l] for i in cities for j in cities for l in possible_hubs))
    
m.setObjective(gp.quicksum(gp.quicksum(u.at[i,k] * z[(i,k)] for k in possible_hubs) + gp.quicksum(alpha*u.at[k,l]*y[(i,k,l)] for k in possible_hubs for l in possible_hubs) + gp.quicksum(u.at[l,j] * x[i,j,l] for l in possible_hubs for j in cities) for i in cities))

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 551206 rows, 1069524 columns and 3195288 nonzeros
Model fingerprint: 0x0d792a4f
Variable types: 1069443 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e-04, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+07]
Presolve removed 6642 rows and 13122 columns
Presolve time: 3.39s
Presolved: 544564 rows, 1056402 columns, 3175605 nonzeros
Variable types: 1056321 continuous, 81 integer (81 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.36s

Barrier statistics:
 Dense cols : 81
 AA' NZ     : 2.314e+06
 Factor NZ  : 4.254e+06 (roughly 600 MB of memory)
 Factor Ops : 3.825e+08 (less than 1 second per iteration)
 Threads    : 5

             


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7655746.87    0    3          - 7655746.87      -     -   87s
H    0     0                    1.065475e+07 7655746.87  28.1%     -   88s
H    0     0                    9974369.5122 7655746.87  23.2%     -   89s
     0     0 7796970.98    0    5 9974369.51 7796970.98  21.8%     -  103s
     0     0 7797848.46    0    5 9974369.51 7797848.46  21.8%     -  107s
     0     0 7801281.41    0    2 9974369.51 7801281.41  21.8%     -  110s
     0     0 7801314.81    0    2 9974369.51 7801314.81  21.8%     -  113s
     0     0 7803841.96    0    2 9974369.51 7803841.96  21.8%     -  120s
     0     0 7804803.02    0    2 9974369.51 7804803.02  21.8%     -  123s
     0     0 7804843.89    0    2 9974369.51 7804843.89  21.8%     -  124s
     0     0 7804886.41    0    2 9974369.51 7804886.41  21.8%     -  124s
     0     0 7804888.52

#### Open hubs

In [8]:
for key in h.keys():
    if h[key].X != 0:
        print(key)

ANKARA
DIYARBAKIR
ERZURUM
GAZIANTEP
ISTANBUL
IZMIR


### Solution with shortest paths

In [9]:
#model
m = gp.Model("q2_part2")
m.ModelSense = GRB.MINIMIZE
#decision variables 
h=m.addVars(possible_hubs, name="h", vtype=GRB.BINARY)
x=m.addVars(cities, cities, possible_hubs, name="x",vtype=GRB.CONTINUOUS, lb = 0)
y=m.addVars(cities, possible_hubs, possible_hubs, name="y", vtype=GRB.CONTINUOUS, lb = 0)
z=m.addVars(cities, possible_hubs, name="z", vtype=GRB.CONTINUOUS, lb = 0)

In [10]:
hub_limit=m.addConstr(gp.quicksum(fixed_hub_cost[i]*h[i] for i in possible_hubs) <= 2400,"hub_cost_limit")

cons1 = m.addConstrs(
    (gp.quicksum(z[(i,k)] for k in possible_hubs) == o[i] for i in cities))

cons2 = m.addConstrs(
    (gp.quicksum(x[(i,j,l)] for l in possible_hubs) == flow.at[i,j] for i in cities for j in cities))

cons3 = m.addConstrs(
    (gp.quicksum(y[(i,k,l)] for l in possible_hubs) + 
     gp.quicksum(x[(i,j,k)] for j in cities) - 
     gp.quicksum(y[(i,l,k)] for l in possible_hubs) -
     z[(i,k)] == 0 for i in cities for k in possible_hubs))

cons4 = m.addConstrs(
    (z[(i,k)] <= o[i] * h[k] for i in cities for k in possible_hubs))

cons5 = m.addConstrs(
    (x[i,j,l] <= flow.at[i,j] * h[l] for i in cities for j in cities for l in possible_hubs))
    
m.setObjective(gp.quicksum(gp.quicksum(u.at[i,k] * z[(i,k)] for k in possible_hubs) + gp.quicksum(alpha*u_hub.at[k,l]*y[(i,k,l)] for k in possible_hubs for l in possible_hubs) + gp.quicksum(u.at[l,j] * x[i,j,l] for l in possible_hubs for j in cities) for i in cities))

In [11]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 551206 rows, 1069524 columns and 3195288 nonzeros
Model fingerprint: 0x8ff67f9c
Variable types: 1069443 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e-04, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+07]
Presolve removed 6642 rows and 13122 columns
Presolve time: 3.29s
Presolved: 544564 rows, 1056402 columns, 3175605 nonzeros
Variable types: 1056321 continuous, 81 integer (81 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.36s

Barrier statistics:
 Dense cols : 81
 AA' NZ     : 2.382e+06
 Factor NZ  : 4.254e+06 (roughly 700 MB of memory)
 Factor Ops : 3.825e+08 (less than 1 second per iteration)
 Threads    : 5

             

  96   1.47330046e+07  3.71082254e+06  4.19e-03 1.37e-04  5.20e+00    99s
  97   1.43226616e+07  3.84082166e+06  3.95e-03 1.32e-04  4.95e+00    99s
  98   1.37472150e+07  3.96169381e+06  3.60e-03 1.28e-04  4.62e+00   100s
  99   1.34485379e+07  4.08981345e+06  3.43e-03 1.23e-04  4.42e+00   101s

Barrier performed 99 iterations in 100.95 seconds (51.76 work units)
Barrier solve interrupted - model solved by another algorithm

Concurrent spin time: 64.99s (can be avoided by choosing Method=3)

Solved with dual simplex

Root relaxation: objective 6.799391e+06, 38774 iterations, 94.70 seconds (44.94 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6799391.23    0    1          - 6799391.23      -     -  101s
H    0     0                    6986410.7891 6799391.23  2.68%     -  102s
     0     0 6800626.28    0    3 6986410.79 6800626.28  2.66%     -  114s
     0  

#### Open hubs


In [18]:
for key in y.keys():
    if y[key].X > 1:
        if h[key[1]].X == 0 or h[key[2]].X == 0:
            print("{:>50} {:>15} {:>4} {:>4}".format(str(key), round(y[key].X,3), h[key[1]].X, h[key[2]].X))

           ('ADIYAMAN', 'DIYARBAKIR', 'GAZIANTEP')      371790.463  1.0 -0.0
                ('ADIYAMAN', 'GAZIANTEP', 'ADANA')       73949.277 -0.0  1.0
             ('ADIYAMAN', 'GAZIANTEP', 'ISTANBUL')      297841.186 -0.0  1.0
               ('AGRI', 'DIYARBAKIR', 'SANLIURFA')      180224.828  1.0 -0.0
                    ('AGRI', 'SANLIURFA', 'ADANA')      180224.828 -0.0  1.0
                ('ANTALYA', 'ANKARA', 'GAZIANTEP')      232538.946  1.0 -0.0
            ('ANTALYA', 'GAZIANTEP', 'DIYARBAKIR')      232538.946 -0.0  1.0
                ('ARTVIN', 'ERZURUM', 'GAZIANTEP')       65095.689  1.0 -0.0
                  ('ARTVIN', 'GAZIANTEP', 'ADANA')       65095.689 -0.0  1.0
                   ('AYDIN', 'ADANA', 'GAZIANTEP')      127079.394  1.0 -0.0
              ('AYDIN', 'GAZIANTEP', 'DIYARBAKIR')      127079.394 -0.0  1.0
             ('BINGOL', 'DIYARBAKIR', 'GAZIANTEP')      176706.507  1.0 -0.0
             ('BINGOL', 'DIYARBAKIR', 'SANLIURFA')       29914.539  1.0 -0.0

In [22]:
for key in h.keys():
    if h[key].X > 0:
        print(key)

ADANA
ANKARA
DIYARBAKIR
ERZURUM
ISTANBUL
IZMIR


In [19]:
u_hub.at["DIYARBAKIR", "GAZIANTEP"] + u_hub.at["GAZIANTEP","ADANA"]

0.0301398050512601

In [21]:
u_hub.at["DIYARBAKIR", "ADANA"]

0.0301398050512601